In [15]:
import cv2
import numpy as np

def remove_three_borders_with_dilation(image_path, output_path):
    """
    階層構造を考慮して3つの枠を検出し、膨張処理を加えて確実に除去する関数
    """
    # 画像の読み込み
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("画像を読み込めませんでした")

    # 画像のコピーを作成
    result = img.copy()
    debug_img = img.copy()
    
    # グレースケールに変換
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 二値化処理
    _, binary = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV)
    
    # 輪郭検出（階層構造を保持）
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # 画像の面積を計算
    image_area = img.shape[0] * img.shape[1]
    
    def is_rectangle(contour, min_area_ratio):
        area = cv2.contourArea(contour)
        if area < image_area * min_area_ratio:
            return False
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        return len(approx) == 4
    
    # 枠の候補を保存
    border_candidates = []
    
    if hierarchy is not None:
        hierarchy = hierarchy[0]
        for i, (contour, h) in enumerate(zip(contours, hierarchy)):
            if is_rectangle(contour, 0.01):
                area = cv2.contourArea(contour)
                border_candidates.append({
                    'contour': contour,
                    'area': area,
                    'parent': h[3],
                    'index': i
                })
    
    # 面積でソート
    border_candidates.sort(key=lambda x: x['area'], reverse=True)
    
    # 最大3つの適切な枠を選択
    selected_borders = []
    used_indices = set()
    
    for candidate in border_candidates:
        if candidate['index'] in used_indices:
            continue
        if candidate['parent'] != -1 and candidate['parent'] in used_indices:
            continue
        selected_borders.append(candidate['contour'])
        used_indices.add(candidate['index'])
        if len(selected_borders) == 3:
            break
    
    # デバッグ用に検出した枠を描画
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    for i, border in enumerate(selected_borders):
        color = colors[i % len(colors)]
        cv2.drawContours(debug_img, [border], -1, color, 2)
        M = cv2.moments(border)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            area = cv2.contourArea(border)
            cv2.putText(debug_img, f"{i+1}: {area:.0f}", (cx-50, cy), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # デバッグ画像を保存
    cv2.imwrite('detected_borders_final.jpg', debug_img)
    
    # 枠を除去する関数
    def remove_border(img, contour, kernel_size=3, iterations=1):
        # マスクを作成
        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, 2)
        
        # マスクを膨張させる
        kernel = np.ones((kernel_size, kernel_size), np.uint8)
        dilated_mask = cv2.dilate(mask, kernel, iterations=iterations)
        
        # マスク領域を白で塗りつぶす
        img[dilated_mask > 0] = [255, 255, 255]
    
    # 検出した各枠に対して除去処理を実行
    for border in selected_borders:
        # 外側の枠
        if cv2.contourArea(border) > image_area * 0.5:
            remove_border(result, border, kernel_size=5, iterations=2)
        # 内側の枠
        else:
            remove_border(result, border, kernel_size=3, iterations=1)
    
    # 結果を保存
    cv2.imwrite(output_path, result)
    
    print(f"検出された枠の数: {len(selected_borders)}")
    for i, border in enumerate(selected_borders):
        area = cv2.contourArea(border)
        print(f"枠 {i+1}: 面積 = {area:.0f}")
    
    return result

def main():
    input_path = '../test_data/input/test_1.jpg'
    output_path = '../test_data/input/test_2.png'
    
    try:
        result = remove_three_borders_with_dilation(input_path, output_path)
        print(f"処理が完了しました。")
        print(f"結果画像: {output_path}")
        print(f"検出された枠の確認用画像: detected_borders_final.jpg")
        
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")

if __name__ == "__main__":
    main()

検出された枠の数: 3
枠 1: 面積 = 1108016
枠 2: 面積 = 471717
枠 3: 面積 = 219584
処理が完了しました。
結果画像: ../test_data/input/test_2.png
検出された枠の確認用画像: detected_borders_final.jpg
